In [1]:
# @inproceedings{karkkainenfairface,
#   title={FairFace: Face Attribute Dataset for Balanced Race, Gender, and Age for Bias Measurement and Mitigation},
#   author={Karkkainen, Kimmo and Joo, Jungseock},
#   booktitle={Proceedings of the IEEE/CVF Winter Conference on Applications of Computer Vision},
#   year={2021},
#   pages={1548--1558}
# }

In [1]:
import pandas as pd
import dlib

In [53]:
fairface_train = pd.read_csv("fairface_label_train.csv")
fairface_val = pd.read_csv("fairface_label_val.csv")

In [84]:
def give_filenames(dataset):
    male_state = (dataset['age'] == '20-29') & (dataset['gender'] == 'Male') 
    female_state = (dataset['age'] == '20-29') & (dataset['gender'] == 'Female')
    male = dataset[male_state][:2001]
    female = dataset[female_state][:2001]
    
    male_filenames = []
    for i in range(len(male)):
        male_filenames.append(male.iloc[i]['file'])
        
    female_filenames = []
    for i in range(len(female)):
        female_filenames.append(female.iloc[i]['file'])
    return male_filenames, female_filenames

In [55]:
male_filenames, female_filenames = give_filenames(fairface_train)

## For Training Dataset

In [57]:
# Split training dataset to male and female

import os
import shutil
for file in female_filenames:
    filename_from = os.path.join('fairface-img-margin025-trainval/', file)
    filename_to = os.path.join('female/', file[6:])  #     os.rename(filename_from, filename_to)
    try:
        if (os.path.exists(filename_from)):
            shutil.move(filename_from, filename_to)
            os.replace(filename_from, filename_to)
            filein.close()
    except:
        continue

## For Testing Dataset

In [58]:
male_fn, female_fn = give_filenames(fairface_val)

In [60]:
# Split val dataset to male and female

import os
import shutil

for file in male_fn:
    filename_from = os.path.join('fairface-img-margin025-trainval/', file)
    filename_to = os.path.join('male/', file[4:])
#     os.rename(filename_from, filename_to)
    try:
        if (os.path.exists(filename_from)):
            shutil.move(filename_from, filename_to)
            os.replace(filename_from, filename_to)
            filein.close()
    except:
        continue

## Make Up Code

In [61]:
from pathlib import Path

In [88]:
from PIL import Image, ImageDraw
import face_recognition

def save_makeup_pic(filename):
    # Load the jpg file into a numpy array
    image = face_recognition.load_image_file("train_data/female/" + filename[6:])   # Change this to male or female

    # Find all facial features in all the faces in the image
    face_landmarks_list = face_recognition.face_landmarks(image)

    pil_image = Image.fromarray(image)
    for face_landmarks in face_landmarks_list:
        d = ImageDraw.Draw(pil_image, 'RGBA')

        # Make the eyebrows into a nightmare
        d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
        d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
        d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
        d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

        # Gloss the lips
        d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
        d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
        d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
        d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

        # Sparkle the eyes
        d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
        d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

        # Apply some eyeliner
        d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
        d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)
        fn = Path('female/makeup_' + filename[6:])
        if not fn.is_dir():
            pil_image.save('female/makeup_' + filename[6:])    # Change this to male or female

## Make Up for Male Training Dataset

In [89]:
male_fn, female_fn = give_filenames(fairface_train)

In [90]:
# For generated makeup dataset
for fn in female_fn:
    save_makeup_pic(fn)